In [1]:
import os
import glob
import json
import pandas as pd
from flatten_json import flatten
import re
import geopandas as gpd
import geopandas as gpd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import logging
from typing import Any, Dict, List, Optional, TextIO, Union

In [2]:
folder_path = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output'

# Function to extract the 'yyyy-mm' from the filename
def extract_timeperiod(filename):
    basename = os.path.basename(filename)
    return basename[:7]  # Assumes the format 'yyyy-mm' at the start of the filename

In [3]:
import json
import os
import re
from functools import reduce
from typing import Any, Callable, Dict, List, Optional, Union

import pandas as pd

# -----------------------------------------------------------------------------
# Generic parsing utilities
# -----------------------------------------------------------------------------

def parse_paren_text(text: str) -> List[str]:
    """
    Extract substrings inside parentheses, e.g. '(A | 1), (B | 2)' -> ['A | 1', 'B | 2']
    """
    return re.findall(r"\(\s*(.*?)\s*\)", text or "")


def extract_from_list(
    data_list: List[Dict[str, Any]],
    detail_key: str,
    value_name: str,
    cast_fn: Callable[[str], Any] = lambda x: x
) -> pd.DataFrame:
    """
    Extracts (revenue_circle | value) entries from each dict in data_list under detail_key.
    Returns DataFrame with columns ['district','revenue_circle',value_name], even if empty.
    """
    records = []
    for entry in data_list:
        district = entry.get('district')
        text = entry.get(detail_key, "")
        for item in parse_paren_text(text):
            parts = [p.strip() for p in item.split('|', 1)]
            if len(parts) != 2:
                continue
            rev_circle, raw_val = parts
            try:
                val = cast_fn(raw_val)
            except Exception:
                continue
            records.append({
                'district': district,
                'revenue_circle': rev_circle,
                value_name: val
            })
    df = pd.DataFrame(records)
    # ensure columns exist for outer joins
    for col in ['district', 'revenue_circle', value_name]:
        if col not in df.columns:
            df[col] = pd.Series(dtype='object')
    return df

# -----------------------------------------------------------------------------
# Specialized extractors
# -----------------------------------------------------------------------------

def extract_from_dict_of_lists(
    data_dict: Dict[str, List[Dict[str, Any]]]
) -> pd.DataFrame:
    """
    Flatten a dict of lists (e.g., infrastructure indicators) into wide table.
    """
    rows = []
    for indicator, entries in data_dict.items():
        for entry in entries:
            district = entry.get('district')
            for detail in entry.get('details', []):
                block_text = detail.get('block', '')
                for item in parse_paren_text(block_text):
                    parts = [p.strip() for p in item.split('|', 1)]
                    if len(parts) != 2:
                        continue
                    rev_circle, raw_val = parts
                    try:
                        val = int(raw_val)
                    except ValueError:
                        continue
                    rows.append({
                        'district': district,
                        'revenue_circle': rev_circle,
                        indicator: val
                    })
    df = pd.DataFrame(rows)
    if df.empty:
        # no data, return empty wide
        cols = ['district','revenue_circle'] + list(data_dict.keys())
        return pd.DataFrame(columns=cols)
    # pivot to wide form
    return (
        df
        .groupby(['district','revenue_circle'], as_index=False)
        .agg({ind: 'sum' for ind in data_dict.keys()})
    )


def extract_human_lives(
    huma_dict: Dict[str, Any],
    detail_key: str = 'details'
) -> pd.DataFrame:
    """
    Extract lives_lost_confirmed and lives_lost_missing.
    """
    df_confirmed = extract_from_list(huma_dict.get('confirmed', []), detail_key, 'lives_lost_confirmed', cast_fn=int)
    df_missing   = extract_from_list(huma_dict.get('missing',   []), detail_key, 'lives_lost_missing',   cast_fn=int)
    # merge, ensure columns exist
    df = pd.merge(df_confirmed, df_missing, on=['district','revenue_circle'], how='outer')
    for col in ['lives_lost_confirmed','lives_lost_missing']:
        if col not in df.columns:
            df[col] = 0
    return df

def extract_relief_inmates(
    data_list: List[Dict[str, Any]],
    detail_key: str = 'details',
    value_name: str = 'relief_inmates'
) -> pd.DataFrame:
    """
    Extracts (revenue_circle | inmate_count) entries from each dict in data_list under detail_key.
    Returns DataFrame with columns ['district','revenue_circle',value_name].
    """
    # Reuse generic extractor, casting to int
    df = extract_from_list(data_list, detail_key, value_name, cast_fn=lambda x: int(re.search(r"\d+", x).group()))
    return df


# -----------------------------------------------------------------------------
# Main JSON extraction
# -----------------------------------------------------------------------------

def extract_json(
    data_source: Union[str, Any],
    timeperiod: str
) -> Optional[pd.DataFrame]:
    """
    Load JSON, extract population, crop, human lives, infra and relief data.
    """
    try:
        if isinstance(data_source, str):
            with open(data_source, 'r', encoding='utf-8') as f:
                data = json.load(f)
        else:
            data = json.load(data_source)
    except Exception as e:
        print(f"Error loading JSON: {e}")
        return None

    # Pop & Crop
    ap = data.get("affectedPopulation", [])
    pop_records  = []
    crop_records = []

    for entry in ap:
        district = entry.get('district')
        details  = entry.get('details', '')
        # find all "(...)" chunks
        for chunk in re.findall(r'\(\s*(.*?)\s*\)', details):
            parts = [p.strip() for p in chunk.split('|')]
            # we expect exactly 3 parts: [rev_circle, pop, crop]
            if len(parts) != 3:
                continue

            rev_circle, pop_raw, crop_raw = parts

            # parse population
            pop_match = re.search(r'\d+', pop_raw)
            if pop_match:
                pop_records.append({
                    'district':           district,
                    'revenue_circle':     rev_circle,
                    'Population Affected': int(pop_match.group())
                })

            # parse crop area
            crop_match = re.search(r'\d+(?:\.\d+)?', crop_raw)
            if crop_match:
                crop_records.append({
                    'district':       district,
                    'revenue_circle': rev_circle,
                    'Crop Area':      float(crop_match.group())
                })

    # build DataFrames with explicit columns to avoid KeyErrors
    df_pop  = pd.DataFrame(pop_records,  columns=['district','revenue_circle','Population Affected'])
    df_crop = pd.DataFrame(crop_records, columns=['district','revenue_circle','Crop Area'])

    # now merge—both dfs will have the correct keys
    df_pop_crop = pd.merge(
        df_pop, df_crop,
        on=['district','revenue_circle'],
        how='outer'
    ).fillna(0)


    # Human lives
    df_hll = extract_human_lives(data.get('hllDetails', {}))

    # Infrastructure
    df_inf = extract_from_dict_of_lists(data.get('infDamageDetails', {}))

    # Relief camps & centers
    relief = data.get('reliefCampsAndCenters', [])
    records = []
    for entry in relief:
        district = entry.get('district')
        for key in ['reliefCamps', 'reliefCenters']:
            text = entry.get(key, '')
            for item in parse_paren_text(text):
                parts = [p.strip() for p in item.split('|', 1)]
                if len(parts) != 2:
                    continue
                rev_circle, raw_val = parts
                try:
                    val = int(re.search(r"\d+", raw_val).group())
                except Exception:
                    val = 0
                records.append({
                    'district': district,
                    'revenue_circle': rev_circle,
                    key: val
                })
    df_rel = pd.DataFrame(records)
    if not df_rel.empty:
        df_rel = (
            df_rel
            .groupby(['district','revenue_circle'], as_index=False)
            .agg({'reliefCamps':'sum','reliefCenters':'sum'})
        )
    else:
        df_rel = pd.DataFrame(columns=['district','revenue_circle','reliefCamps','reliefCenters'])
    
    # 5) Relief inmates
    df_inmates = extract_relief_inmates(data.get('campInmates', []), detail_key='details', value_name='relief_inmates')

    # Merge all together
    dfs = [df_pop_crop, df_hll, df_inf, df_rel,df_inmates]
    final_df = reduce(
        lambda left, right: pd.merge(left, right, on=['district','revenue_circle'], how='outer'),
        dfs
    ).fillna(0)

    final_df['timeperiod'] = timeperiod
    return final_df

# -----------------------------------------------------------------------------
# Folder processing
# -----------------------------------------------------------------------------

def extract_folder(
    folder_path: str,
    pattern: str = r"^.+_\d{2}\.json$"
) -> pd.DataFrame:
    all = []
    for fname in os.listdir(folder_path):
        if re.match(pattern, fname):
            tp = os.path.splitext(fname)[0]
            path = os.path.join(folder_path, fname)
            df = extract_json(path, tp)
            if df is not None and not df.empty:
                all.append(df)
    return pd.concat(all, ignore_index=True) if all else pd.DataFrame()

# Example usage:
master_df = extract_folder(folder_path)
master_df# print(master_df.head())


C:\Users\saura\AppData\Local\Temp\ipykernel_51904\2532102533.py:241: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).fillna(0)


,district,revenue_circle,Population Affected,Crop Area,lives_lost_confirmed,lives_lost_missing,embBreached,embAffected,roadAffected,bridgeAffected,reliefCamps,reliefCenters,relief_inmates,timeperiod
0,Biswanath,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05
1,Dima-Hasao,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05
2,Morigaon,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05
3,Nalbari,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05
4,Sivasagar,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,Sribhumi,RK Nagar,0.0,0.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2025_07
1623,Sribhumi,Sribhumi Sadar,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025_07
1624,Udalguri,Khairabari,0.0,0.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025_07
1625,Udalguri,Mazbat,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025_07


In [ ]:
master_df

In [21]:
# Function to flatten the JSON into a dataframe
def extract_json(data, timeperiod, file_path):
    try:
        # Normalize specific fields from JSON and create a dataframe
        data_file = json.load(data)

        affected_pop = pd.DataFrame(data_file["affectedPopulation"])

        data_list = []

        # Iterate over each entry in the 'details' Series
        for index, row in affected_pop.iterrows():
            district = row['district']  # Capture the district for the current row
            details = row['details']  # Capture the details for the current row
            
            # Find matches for the current details string
            match = re.findall(r'\(\s*(.*?)\s*\)', details)
            
            for m in match:
                try:
                    # Split the extracted string by ' | ' to get the revenue circle, population affected, and crop area
                    revenue_circle, population_affected, crop_area = m.split(' | ')
                    population_affected = int(re.search(r'\d+', population_affected).group())
                    crop_area = float(re.search(r'\d+(\.\d+)?', crop_area).group())
                    
                    # Append the results as a dictionary to the list, including the correct district
                    data_list.append({
                        "district": district,  # Add the correct district value here
                        "revenue_circle": revenue_circle.strip(),
                        "Population Affected": population_affected,
                        "Crop Area": crop_area
                    })
                except Exception as e:
                    print("Error processing match:", m, e)

        # Create a DataFrame from the collected data
        pop_crop = pd.DataFrame(data_list)

        # human lives lost
        hll = data_file["hllDetails"]
        df = pd.DataFrame(hll)

        # Function to extract the details column into a structured DataFrame
        def extract_details(df_column, lives_lost_col):
            extracted_data = []
            
            for row in df_column:
                district = row['district']
                details = row['details']
                # Extract the revenue circle and lives lost information
                for match in details.split('), '):
                    match = match.strip('()')
                    revenue_circle, lives_lost = match.split(' | ')
                    extracted_data.append({
                        'district': district,
                        'revenue_circle': revenue_circle.strip(),
                        lives_lost_col: int(lives_lost)
                    })
            
            return pd.DataFrame(extracted_data)

        # Extract lives lost for both 'confirmed' and 'missing'
        df_confirmed = extract_details(df['confirmed'], 'lives_lost_confirmed')
        df_missing = extract_details(df['missing'], 'lives_lost_missing')

        # Merge the two DataFrames on 'district' and 'revenue_circle'
        hll_final = pd.merge(df_confirmed, df_missing, on=['district', 'revenue_circle'], how='outer')

        # INFRASTRUCTURE
        inf = data_file["infDamageDetails"]
        
        extracted_data = []

        # Define the list of indicator types to iterate over
        indicator_types = ["embBreached", "embAffected", "bridgeAffected", "roadAffected"]

        # Iterate through each indicator type in the data
        for indicator in indicator_types:
            if indicator in inf:
                for entry in inf[indicator]:
                    district = entry['district']
                    for detail in entry['details']:
                        # Extract the block name and clean up the indicator value (remove parentheses)
                        revenue_circle = detail['block'].split('|')[0].strip('() ')
                        indicator_value = detail['block'].split('|')[1].strip('() ')
                        
                        # Convert the cleaned value to an integer
                        indicator_value = int(indicator_value)
                        
                        # Append a dictionary with the extracted data
                        extracted_data.append({
                            'district': district,
                            'revenue_circle': revenue_circle,
                            'indicator': indicator,
                            'value': indicator_value
                        })

        # Convert the list to a DataFrame
        inf_dmg = pd.DataFrame(extracted_data)

        # Pivot the table to get one row per block, and each indicator as a separate column
        inf_dmg = inf_dmg.pivot_table(index=['district', 'revenue_circle'], columns='indicator', values='value', fill_value=0).reset_index()

        # Flatten the multi-level column index after pivot
        inf_dmg.columns.name = None  # Remove the index name from columns
        inf_dmg.columns = [col if isinstance(col, str) else col[1] for col in inf_dmg.columns]


        # Combine all dataframes into one
        combined_df = pd.merge(pop_crop, hll_final, on=['revenue_circle', 'district'])
        final_df = pd.merge(combined_df, inf_dmg, on=['revenue_circle', 'district'])

        final_df['timeperiod'] = timeperiod
        
        return final_df

    except KeyError as e:
        print(f"Warning: Missing key {e} in file {file_path}. Skipping this file.")
        return None

all_dataframes = []

# Process each JSON file in the folder
for file_path in glob.glob(os.path.join(folder_path, '*.json')):
    with open(file_path, 'r') as f:
        data = json.load(f)
    data = open(file_path)
    print(file_path)
    timeperiod = extract_timeperiod(file_path)
    # Extract timeperiod from the filename
    
    # Flatten the JSON and process it
    df = extract_json(data, timeperiod, file_path)
    
    # Append the dataframe to the list
    all_dataframes.append(df)

combined_df = pd.concat(all_dataframes, ignore_index=True)

# Save the combined dataframe to a CSV file (optional)
combined_df

D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\2021_05.json
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\2021_06.json
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\2021_07.json
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\2021_08.json
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\2021_09.json
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\2021_10.json
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\2022_04.json
D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\2022_

,district,revenue_circle,Population Affected,Crop Area,lives_lost_confirmed,lives_lost_missing,bridgeAffected,embAffected,embBreached,roadAffected,timeperiod
0,Barpeta,Kalgachia,0,1.73,0,0,0.0,0.0,0.0,0.0,2021_06
1,Barpeta,Barpeta,0,0.00,1,0,0.0,0.0,0.0,0.0,2021_06
2,Barpeta,Chenga,0,0.00,1,0,0.0,0.0,0.0,0.0,2021_06
3,Biswanath,Halem,11,35.50,0,0,0.0,2.0,2.0,2.0,2021_06
4,Biswanath,Gohpur,0,8.50,0,0,0.0,0.0,1.0,0.0,2021_06
...,...,...,...,...,...,...,...,...,...,...,...
1420,Lakhimpur,Bihpuria,0,0.00,0,0,0.0,0.0,0.0,0.0,2024_09
1421,Lakhimpur,Narayanpur,4,0.00,0,0,0.0,0.0,1.0,0.0,2024_09
1422,Sivasagar,Demow,0,76.50,0,0,0.0,0.0,0.0,0.0,2024_09
1423,Sivasagar,Sivsagar,0,89.00,0,0,0.0,0.0,0.0,0.0,2024_09


In [ ]:
combined_df

In [5]:
# Function to flatten the JSON into a dataframe
def extract_json(data, timeperiod):
    # Normalize specific fields from JSON and create a dataframe
    data_file = json.load(data)

    affected_pop = pd.DataFrame(data_file["affectedPopulation"])

    data_list = []

    # Iterate over each entry in the 'details' Series
    for index, row in affected_pop.iterrows():
        district = row['district']  # Capture the district for the current row
        details = row['details']  # Capture the details for the current row
        
        # Find matches for the current details string
        match = re.findall(r'\(\s*(.*?)\s*\)', details)
        
        for m in match:
            try:
                # Split the extracted string by ' | ' to get the revenue circle, population affected, and crop area
                revenue_circle, population_affected, crop_area = m.split(' | ')
                population_affected = int(re.search(r'\d+', population_affected).group())
                crop_area = float(re.search(r'\d+(\.\d+)?', crop_area).group())
                
                # Append the results as a dictionary to the list, including the correct district
                data_list.append({
                    "district": district,  # Add the correct district value here
                    "revenue_circle": revenue_circle.strip(),
                    "Population Affected": population_affected,
                    "Crop Area": crop_area
                })
            except Exception as e:
                print("Error processing match:", m, e)

            # Create a DataFrame from the collected data
    pop_crop = pd.DataFrame(data_list)
    #pop_extracted

    # human lives lost
    hll = data_file["hllDetails"]
    df = pd.DataFrame(hll)

    # Function to extract the details column into a structured DataFrame
    def extract_details(df_column, lives_lost_col):
        extracted_data = []
        
        for row in df_column:
            district = row['district']
            details = row['details']
            # Extract the revenue circle and lives lost information
            for match in details.split('), '):
                match = match.strip('()')
                revenue_circle, lives_lost = match.split(' | ')
                extracted_data.append({
                    'district': district,
                    'revenue_circle': revenue_circle.strip(),
                    lives_lost_col: int(lives_lost)
                })
        
        return pd.DataFrame(extracted_data)

    # Extract lives lost for both 'confirmed' and 'missing'
    df_confirmed = extract_details(df['confirmed'], 'lives_lost_confirmed')
    df_missing = extract_details(df['missing'], 'lives_lost_missing')

    # Merge the two DataFrames on 'district' and 'revenue_circle'
    hll_final = pd.merge(df_confirmed, df_missing, on=['district', 'revenue_circle'], how='outer')

    # INFRASTRUCTURE
    inf = data_file["infDamageDetails"]
    
    #inf = data_file["infDamageDetails"]
    extracted_data = []

    # Define the list of indicator types to iterate over
    indicator_types = ["embBreached", "embAffected", "bridgeAffected", "roadAffected","campInmates"]

    # Iterate through each indicator type in the data
    for indicator in indicator_types:
        if indicator in inf:
            for entry in inf[indicator]:
                district = entry['district']
                for detail in entry['details']:
                    # Extract the block name and clean up the indicator value (remove parentheses)
                    revenue_circle = detail['block'].split('|')[0].strip('() ')
                    indicator_value = detail['block'].split('|')[1].strip('() ')
                    
                    # Convert the cleaned value to an integer
                    indicator_value = int(indicator_value)
                    
                    # Append a dictionary with the extracted data
                    extracted_data.append({
                        'district': district,
                        'revenue_circle': revenue_circle,
                        'indicator': indicator,
                        'value': indicator_value
                    })

    # Convert the list to a DataFrame
    inf_dmg = pd.DataFrame(extracted_data)

    # Pivot the table to get one row per block, and each indicator as a separate column
    inf_dmg = inf_dmg.pivot_table(index=['district', 'revenue_circle'], columns='indicator', values='value', fill_value=0).reset_index()

    # Flatten the multi-level column index after pivot
    inf_dmg.columns.name = None  # Remove the index name from columns
    inf_dmg.columns = [col if isinstance(col, str) else col[1] for col in inf_dmg.columns]
    
    # Add the timeperiod column to each dataframe

    # Combine all dataframes into one
    combined_df = pd.merge(pop_crop,hll_final,on=['revenue_circle','district'])
    final_df = pd.merge(combined_df,inf_dmg,on=['revenue_circle','district'])
    final_df['timeperiod']=timeperiod
    #combined_df = pd.concat([pop_crop, hll_final, inf_dmg,], ignore_index=True)
    
    return final_df


In [14]:
all_dataframes = []

# Process each JSON file in the folder
for file_path in glob.glob(os.path.join(folder_path, '*.json')):
    with open(file_path, 'r') as f:
        data = json.load(f)
    data = open(file_path)
    print(file_path)
    timeperiod = extract_timeperiod(file_path)
    # Extract timeperiod from the filename
    
    # Flatten the JSON and process it
    df = extract_json(data, timeperiod, file_path)
    
    # Append the dataframe to the list
    all_dataframes.append(df)

combined_df = pd.concat(all_dataframes, ignore_index=True)

# Save the combined dataframe to a CSV file (optional)
combined_df#.to_csv(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\combined_loss_damage_data.csv', index=False)

D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\DRIMS_api_output\2021_05.json


AttributeError: 'list' object has no attribute 'pivot_table'

#### Tests

In [27]:
import pandas as pd
import numpy as np
from scipy import stats as stats_module
from sklearn.metrics import cohen_kappa_score


def compare_dataframes(df1: pd.DataFrame, df2: pd.DataFrame,
                       index_cols: list, report_top_n: int = 10) -> dict:
    """
    Compare two DataFrames on specified index columns and common value columns.

    Parameters:
    - df1, df2: DataFrames to compare.
    - index_cols: list of column names to set as index in both DataFrames.
    - report_top_n: number of most problematic rows to report.

    Returns:
    A dictionary containing:
      * column_summary: DataFrame summarizing matches/mismatches per column.
      * row_mismatch_counts: Series counting mismatches per row.
      * stats_summary: DataFrame of statistical tests for numeric/categorical columns.
      * problematic_rows: DataFrame of the top rows with most mismatches.
    """
    # 1. Align on index
    df1_idx = df1.set_index(index_cols)
    df2_idx = df2.set_index(index_cols)

    common_index = df1_idx.index.intersection(df2_idx.index)
    df1_cmp = df1_idx.loc[common_index]
    df2_cmp = df2_idx.loc[common_index]

    # 2. Identify common columns
    common_cols = df1_cmp.columns.intersection(df2_cmp.columns)

    # 3. Compare values and build mismatch report
    col_summary = []
    row_mismatch_counts = {}

    for col in common_cols:
        s1 = df1_cmp[col]
        s2 = df2_cmp[col]
        # element-wise comparison, treating NaNs as equal
        eq = s1.eq(s2) | (s1.isna() & s2.isna())
        total = len(eq)
        matches = int(eq.sum())
        mismatches = total - matches
        mismatch_ratio = mismatches / total if total else np.nan
        # collect indices for mismatches
        idx_mismatches = eq[~eq].index.tolist()
        # accumulate row mismatch counts
        for idx in idx_mismatches:
            row_mismatch_counts[idx] = row_mismatch_counts.get(idx, 0) + 1
        col_summary.append({
            'column': col,
            'total': total,
            'matches': matches,
            'mismatches': mismatches,
            'mismatch_ratio': mismatch_ratio
        })

    column_summary_df = pd.DataFrame(col_summary).set_index('column')

    # 4. Statistical tests
    stats_records = []
    numeric_cols = [c for c in common_cols if np.issubdtype(df1_cmp[c].dtype, np.number)]
    cat_cols = [c for c in common_cols if c not in numeric_cols]

    # Numeric: Pearson correlation + paired t-test
    for col in numeric_cols:
        a = df1_cmp[col].astype(float)
        b = df2_cmp[col].astype(float)
        corr = a.corr(b)
        t_stat, p_val = stats_module.ttest_rel(a, b, nan_policy='omit')
        stats_records.append({
            'column': col,
            'test': 'pearson_corr',
            'value': corr
        })
        stats_records.append({
            'column': col,
            'test': 'paired_t_stat',
            'value': t_stat
        })
        stats_records.append({
            'column': col,
            'test': 'paired_t_pvalue',
            'value': p_val
        })

    # Categorical: Cohen's kappa
    for col in cat_cols:
        a = df1_cmp[col].fillna('NA').astype(str)
        b = df2_cmp[col].fillna('NA').astype(str)
        try:
            kappa = cohen_kappa_score(a, b)
        except Exception:
            kappa = np.nan
        stats_records.append({
            'column': col,
            'test': 'cohen_kappa',
            'value': kappa
        })

    stats_summary_df = pd.DataFrame(stats_records)

    # 5. Identify problematic rows
    row_counts = pd.Series(row_mismatch_counts).sort_values(ascending=False)
    problematic_rows = row_counts.head(report_top_n).rename('mismatch_count')

    # 6. Compile report
    report = {
        'column_summary': column_summary_df,
        'row_mismatch_counts': row_counts,
        'stats_summary': stats_summary_df,
        'problematic_rows': problematic_rows
    }
    return report


if __name__ == '__main__':
    # Example Usage
    # Load dataframes (replace paths with actual file locations)
    df1 = master_df
    df2 = combined_df

    # Specify index columns
    index_columns = ['district', 'revenue_circle', 'timeperiod']

    # Run comparison
    report = compare_dataframes(df1, df2, index_columns)

    # Display summaries
    print("Column Comparison Summary:\n", report['column_summary'])
    print("\nTop Problematic Rows (most mismatches):\n", report['problematic_rows'])
    print("\nStatistical Tests Summary:\n", report['stats_summary'])


Column Comparison Summary:
                       total  matches  mismatches  mismatch_ratio
column                                                          
Population Affected    1425     1425           0             0.0
Crop Area              1425     1425           0             0.0
lives_lost_confirmed   1425     1425           0             0.0
lives_lost_missing     1425     1425           0             0.0
embBreached            1425     1425           0             0.0
embAffected            1425     1425           0             0.0
roadAffected           1425     1425           0             0.0
bridgeAffected         1425     1425           0             0.0

Top Problematic Rows (most mismatches):
 Series([], Name: mismatch_count, dtype: object)

Statistical Tests Summary:
                   column             test  value
0    Population Affected     pearson_corr    1.0
1    Population Affected    paired_t_stat    NaN
2    Population Affected  paired_t_pvalue    NaN
3      

Variable export

In [6]:
rc_gdf = gpd.read_file(r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Maps\Geojson\assam_rc_2024-11.geojson')

In [7]:
rc_gdf

,revenue_ci,revenue_cr,HQ,are_new,dtname,object_id,dtcode11,geometry
0,Gossaigaon (Pt),Gossaigaon (Pt),None,1069,KOKRAJHAR,18-300-00101,18-300,"MULTIPOLYGON (((90.14118 26.74010, 90.15342 26..."
1,Bhowraguri,Bhawraguri,None,159,KOKRAJHAR,18-300-00102,18-300,"MULTIPOLYGON (((90.09811 26.46455, 90.10012 26..."
2,Dotoma,Dotoma,None,304,KOKRAJHAR,18-300-00103,18-300,"MULTIPOLYGON (((90.19819 26.63409, 90.21635 26..."
3,Kokrajhar (Pt),Kokrajhar (Pt),y,990,KOKRAJHAR,18-300-00104,18-300,"MULTIPOLYGON (((90.33590 26.86280, 90.33610 26..."
4,Bagribari (Pt),Bagribari (Pt),None,281,KOKRAJHAR,18-300-00105,18-300,"MULTIPOLYGON (((89.99553 26.35026, 89.99717 26..."
...,...,...,...,...,...,...,...,...
175,Sapekhati,Sapekhati,None,394,CHARAIDEO,18-755-00278,18-755,"MULTIPOLYGON (((95.10936 27.12364, 95.10927 27..."
176,Sonari,Sonari,y,385,CHARAIDEO,18-755-00279,18-755,"MULTIPOLYGON (((94.95742 27.06354, 94.95822 27..."
177,Ujani Majuli,Ujani Majuli,None,322,MAJULI,18-760-00280,18-760,"MULTIPOLYGON (((94.33804 26.89186, 94.33850 26..."
178,Majuli,Majuli,None,648,MAJULI,18-760-00281,18-760,"MULTIPOLYGON (((94.56453 27.18172, 94.56755 27..."


In [8]:
master_df.columns

Index(['district', 'revenue_circle', 'Population Affected', 'Crop Area',
       'lives_lost_confirmed', 'lives_lost_missing', 'embBreached',
       'embAffected', 'roadAffected', 'bridgeAffected', 'reliefCamps',
       'reliefCenters', 'relief_inmates', 'timeperiod'],
      dtype='object')

In [9]:
master_df = master_df.rename(columns={
    'Population Affected': 'Population_affected_Total',
    'Crop Area': 'Crop_Area',
    'bridgeAffected': 'Bridge',
    'lives_lost_confirmed': 'Human_Live_Lost',
    'lives_lost_missing': 'Human_Live_Missing',
    'embBreached': 'Embankment breached',
    'embAffected': 'Embankments affected',
    'roadAffected': 'Roads',
    'reliefCamps': 'Relief Camps',
    'reliefCenters': 'Relief Centers',
    'relief_inmates': 'Relief Inmates'})

In [10]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [11]:
fuzzymatch = fuzzy_merge(rc_gdf, master_df, 'revenue_ci', 'revenue_circle', threshold=80,limit=1)
fuzzymatch

,revenue_ci,revenue_cr,HQ,are_new,dtname,object_id,dtcode11,geometry,matches
0,Gossaigaon (Pt),Gossaigaon (Pt),None,1069,KOKRAJHAR,18-300-00101,18-300,"MULTIPOLYGON (((90.14118 26.74010, 90.15342 26...",Gossaigaon
1,Bhowraguri,Bhawraguri,None,159,KOKRAJHAR,18-300-00102,18-300,"MULTIPOLYGON (((90.09811 26.46455, 90.10012 26...",Bhawraguri
2,Dotoma,Dotoma,None,304,KOKRAJHAR,18-300-00103,18-300,"MULTIPOLYGON (((90.19819 26.63409, 90.21635 26...",Dotma
3,Kokrajhar (Pt),Kokrajhar (Pt),y,990,KOKRAJHAR,18-300-00104,18-300,"MULTIPOLYGON (((90.33590 26.86280, 90.33610 26...",Kokrajhar
4,Bagribari (Pt),Bagribari (Pt),None,281,KOKRAJHAR,18-300-00105,18-300,"MULTIPOLYGON (((89.99553 26.35026, 89.99717 26...",Bagribari
...,...,...,...,...,...,...,...,...,...
175,Sapekhati,Sapekhati,None,394,CHARAIDEO,18-755-00278,18-755,"MULTIPOLYGON (((95.10936 27.12364, 95.10927 27...",Sapekhati
176,Sonari,Sonari,y,385,CHARAIDEO,18-755-00279,18-755,"MULTIPOLYGON (((94.95742 27.06354, 94.95822 27...",Sonari
177,Ujani Majuli,Ujani Majuli,None,322,MAJULI,18-760-00280,18-760,"MULTIPOLYGON (((94.33804 26.89186, 94.33850 26...",Ujani Majuli
178,Majuli,Majuli,None,648,MAJULI,18-760-00281,18-760,"MULTIPOLYGON (((94.56453 27.18172, 94.56755 27...",Majuli


In [12]:
master_df['district_2'] = master_df['district'].str.upper()
master_df

,district,revenue_circle,Population_affected_Total,Crop_Area,Human_Live_Lost,Human_Live_Missing,Embankment breached,Embankments affected,Roads,Bridge,Relief Camps,Relief Centers,Relief Inmates,timeperiod,district_2
0,Biswanath,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05,BISWANATH
1,Dima-Hasao,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05,DIMA-HASAO
2,Morigaon,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05,MORIGAON
3,Nalbari,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05,NALBARI
4,Sivasagar,null,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2021_05,SIVASAGAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,Sribhumi,RK Nagar,0.0,0.0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2025_07,SRIBHUMI
1623,Sribhumi,Sribhumi Sadar,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025_07,SRIBHUMI
1624,Udalguri,Khairabari,0.0,0.0,0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2025_07,UDALGURI
1625,Udalguri,Mazbat,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2025_07,UDALGURI


In [13]:
# Example: Drop rows where all values in the specified subset are zero
subset_columns = ['Population_affected_Total', 'Crop_Area',
       'Bridge', 'Embankment breached', 'Embankments affected',
       'Roads', 'Human_Live_Lost','Relief Camps','Relief Centers','Relief Inmates']
filtered_df = master_df[(master_df[subset_columns] != 0).any(axis=1)]

In [14]:
filtered_df.columns

Index(['district', 'revenue_circle', 'Population_affected_Total', 'Crop_Area',
       'Human_Live_Lost', 'Human_Live_Missing', 'Embankment breached',
       'Embankments affected', 'Roads', 'Bridge', 'Relief Camps',
       'Relief Centers', 'Relief Inmates', 'timeperiod', 'district_2'],
      dtype='object')

In [15]:
rc_complete_matched = fuzzymatch.merge(filtered_df, left_on=['matches','dtname'], right_on=['revenue_circle','district_2'], how='outer')#.to_csv('frims_rc_id_mapping.csv', index=False)
rc_complete_matched = rc_complete_matched[['revenue_ci', 'object_id', 'dtname', #'revenue_cr',
       #'HQ', 'area', 'are_new', 'geometry', 'matches', 'district','revenue_circle', 
       'district_2',
       'Population_affected_Total', 'Crop_Area',
       'timeperiod', 'Bridge', 'Embankment breached', 'Embankments affected',
       'Roads', 'Human_Live_Lost','Relief Camps','Relief Centers','Relief Inmates'
       ]]
df_cleaned=rc_complete_matched.dropna(subset=['timeperiod','object_id'])
df_cleaned = df_cleaned[['district_2','revenue_ci','object_id','dtname','timeperiod','Bridge','Embankment breached','Embankments affected','Roads','Human_Live_Lost','Population_affected_Total', 'Crop_Area','Relief Camps','Relief Centers','Relief Inmates']]
df_cleaned = df_cleaned.rename(columns={'district_2':'DISTRICT'})
df_cleaned

,DISTRICT,revenue_ci,object_id,dtname,timeperiod,Bridge,Embankment breached,Embankments affected,Roads,Human_Live_Lost,Population_affected_Total,Crop_Area,Relief Camps,Relief Centers,Relief Inmates
13,DHUBRI,Agamoni,18-301-00110,DHUBRI,2022_06,0.0,0.0,0.0,9.0,0.0,1190.0,893.5,0.0,0.0,0.0
14,DHUBRI,Agamoni,18-301-00110,DHUBRI,2022_07,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
15,DHUBRI,Agamoni,18-301-00110,DHUBRI,2022_08,0.0,0.0,0.0,0.0,0.0,0.0,35.5,0.0,0.0,0.0
16,DHUBRI,Agamoni,18-301-00110,DHUBRI,2023_06,0.0,0.0,0.0,0.0,0.0,576.0,128.5,1.0,0.0,193.0
17,DHUBRI,Agamoni,18-301-00110,DHUBRI,2023_07,0.0,0.0,0.0,3.0,0.0,3336.0,60.0,1.0,0.0,348.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1436,MAJULI,Ujani Majuli,18-760-00280,MAJULI,2023_08,0.0,0.0,0.0,3.0,0.0,12852.0,443.0,0.0,19.0,0.0
1437,MAJULI,Ujani Majuli,18-760-00280,MAJULI,2023_09,0.0,0.0,0.0,0.0,0.0,0.0,269.0,0.0,0.0,0.0
1438,MAJULI,Ujani Majuli,18-760-00280,MAJULI,2024_06,0.0,0.0,0.0,0.0,0.0,12363.0,0.0,0.0,0.0,0.0
1439,MAJULI,Ujani Majuli,18-760-00280,MAJULI,2024_07,0.0,0.0,3.0,3.0,0.0,14137.0,2123.0,2.0,16.0,91.0


In [16]:
indicator_columns = ['Population_affected_Total', 'Crop_Area',
      'Bridge', 'Embankment breached', 'Embankments affected',
       'Roads', 'Human_Live_Lost','Relief Camps','Relief Centers','Relief Inmates']
                     #'Total_House_Fully_Damaged', 'Total_Animal_Affected']

variable_path = r'D:\CivicDataLab_IDS-DRR\IDS-DRR_Github\Deployment\flood-data-ecosystem-Assam\Sources\DRIMS\data\variables/'

# Step 2: Loop through each indicator column
for indicator in indicator_columns:
    # Step 3: Get unique time periods
    for timeperiod in df_cleaned['timeperiod'].unique():
        # Step 4: Filter the DataFrame by time period and the current indicator column
        filtered_df = df_cleaned[['object_id', indicator]][df_cleaned['timeperiod'] == timeperiod]
        # Step 5: Create directory if it doesn't exist
        if not os.path.exists(indicator):
            os.makedirs(indicator)
        
        # Step 6: Construct file name and save the filtered DataFrame to CSV
        filename = f"{indicator}_{timeperiod}.csv"
        file_path = os.path.join(variable_path,indicator, filename)
        filtered_df.to_csv(file_path, index=False)
